In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("product_sales.csv")
df.head()

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales
0,1,95,92,0.6,0.323,0.08906
1,2,37,27,0.8,0.106,0.01991
2,3,34,26,0.6,0.328,0.03778
3,4,32,30,0.7,0.000,0.08562
4,5,62,45,1.0,0.079,0.02637


In [3]:
perc =[.20, .40, .60, .80]
include =['object', 'float', 'int']
desc = df.describe(percentiles = perc, include = include)
desc

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,125.500000,60.620000,51.596000,1.500400,0.136608,0.050986
std,72.312977,28.318959,24.182223,0.839511,0.157928,0.029084
min,1.000000,10.000000,9.000000,0.100000,0.000000,0.000370
20%,50.800000,33.800000,28.000000,0.600000,0.000000,0.020262
40%,100.600000,49.000000,43.000000,1.200000,0.005000,0.043162
50%,125.500000,57.000000,50.500000,1.400000,0.069000,0.051620
60%,150.400000,69.000000,57.400000,1.800000,0.135000,0.061298
80%,200.200000,89.000000,75.000000,2.300000,0.288400,0.080604
max,250.000000,110.000000,105.000000,3.100000,0.615000,0.099430


## Assumptions
-- Apply price reduction rules based on mean value of incr_cvr and incr_sales
-- Set initial price delta to 0.199

In [20]:
def calc_profit(price_change_val, price_reduction_flag=True):
    original_unit_price = df['unit_price']
    #increment => {-0.1,0.2}
    price_change_multiplier = 1 + price_change_val
    #For iterantion#1 increment = 0, price_change_multiplier = 1
    adjusted_price =  original_unit_price * price_change_multiplier
    
    #print('Original Net price ',np.sum(original_unit_price),'Adjusted Net price',np.sum(adjusted_price))
    
    original_units_sold=df['units_sold'] # Avg units sold at base price in one transaction
    perc_incr_units_sold = 0 #For iteration#1 perc_incr_units_sold = 0
    
    #Calculate additional units for given % of price reduction
    incr_sales = df['incr_sales']
    incr_cvr = df['incr_cvr']
    incr_cvr_default = 0.05
        
    if (price_reduction_flag):
        #consider incr_sales only for price reduction
        perc_incr_units_sold = incr_sales * abs(price_change_val) + incr_cvr * abs(price_change_val)
    else:
        perc_incr_units_sold = incr_cvr_default * abs(price_change_val)
        
    sales_multipler = 1+perc_incr_units_sold
    adjusted_volume = original_units_sold*(sales_multipler)
    
    #print('sales_multipler ',sales_multipler)
    
    #print('Original total units sold ',np.sum(original_units_sold),'Adjusted total units sold',np.sum(adjusted_volume))
    
    original_cost = df['unit_cost'] #unit cost
    
    profit_wo_price_change_effect= np.sum(original_unit_price*original_units_sold)-np.sum(original_cost*original_units_sold)
    
    #For every 10% reduction in price, there may be a some increase in conversion leading to increase in profit
    
    
    #profit_multiplier = 1
     #For iterantion#1 increment =0, profit_multiplier = 1
    #if (price_reduction_flag):
        #consider incr_cvr only for price reduction
    #    profit_multiplier = np.prod(1+(incr_cvr * abs(price_change_val)))
    #else:
        #profit_multiplier = np.prod(1+(incr_cvr_default*abs(price_change_val)))
    
    #print('profit_multiplier ',profit_multiplier)
    
    #net_profit = np.sum(profit_wo_price_change_effect * profit_multiplier)
    
    profit_with_price_change_effect= np.sum(adjusted_price*adjusted_volume)-np.sum(original_cost*adjusted_volume)
    
    #print('profit_wo_price_change_effect ',profit_wo_price_change_effect,'profit_with_price_change_effect',profit_with_price_change_effect)
    
    
    return profit_wo_price_change_effect, profit_with_price_change_effect

In [18]:
# Approach 1: check price elasticity effect on profit for every item
# Approach 2: Apply price elasticity effect on all items before calculating profit

def calc_price_reduction(initial_reductions, incr_cvr_array,incr_sales_array,initial_profit):
    #print('initial reductions =',initial_reductions)
    print('initial profit =',initial_profit)
    intermidiate_profit = 0
    final_profit = 0
    decrements = initial_reductions
    
    for i in range(0,250):  
        if decrements[i] > -0.08:
            boost = 0.02
            decrements[i] = decrements[i]-boost
            profit_wo_price_change_effect, intermidiate_profit = calc_profit(decrements, True)
        
        if (intermidiate_profit < initial_profit) and decrements[i] < 0.19:
            boost = 0.01
            decrements[i] = decrements[i]+boost
            profit_wo_price_change_effect, intermidiate_profit = calc_profit(decrements, False)
   
    if intermidiate_profit > initial_profit:
        final_profit = intermidiate_profit
        final_reductions = decrements
    else:
        final_profit = initial_profit
        final_reductions = decrements
        
    print('final profit = ',final_profit)
    return final_reductions, final_profit

In [6]:
incr_cvr_array = df['incr_cvr'].to_numpy()
np.mean(incr_cvr_array)

0.136608

In [7]:
incr_sales_array = df['incr_sales'].to_numpy()
np.mean(incr_sales_array)

0.05098572

In [13]:
price_change_default_val = np.array(df.shape[0] * [0.0])

In [21]:
converged = False
current_price_change = price_change_default_val
#converged = true is a condition when the price_change vector remains the same
print("====================================")
profit_wo_price_change_effect, current_profit = calc_profit(current_price_change,False)

print("Original profit", profit_wo_price_change_effect)

while not converged:
   # print(‘Initial overall profit:’,find_rev(increment_i))
    print("====================================")
    next_price_change, next_profit = calc_price_reduction(current_price_change,incr_cvr_array,incr_sales_array,current_profit)
    if(current_profit == next_profit):
        converged = True
        break
    else:
        current_profit = next_profit
        current_price_change = next_price_change
    
print('Old Net profit',current_profit) 
print('New Net profit',next_profit) 

        
#print('Price changes', current_price_change)
#print('Old net profit',calc_profit(increment)) 
#price = increment_i
#write.csv(price,"price.csv")

Original profit 3285.899999999998
initial profit = 1489.7662094999505
final profit =  1498.4426105375533
initial profit = 1498.4426105375533
final profit =  1498.679542999951
initial profit = 1498.679542999951
final profit =  1498.7924614999502
initial profit = 1498.7924614999502
final profit =  1498.8613667351528
initial profit = 1498.8613667351528
final profit =  1500.279106499951
initial profit = 1500.279106499951
final profit =  1501.432393499952
initial profit = 1501.432393499952
final profit =  1501.432393499952
Old Net profit 1501.432393499952
New Net profit 1501.432393499952
